## Imports

In [47]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score

## Convert seperate CSVs into one and make sure that frames where no workout is predicted will have it's labels turned to stationary

In [48]:
file_names = ["nothing0", "nothing1", "nothing2", "ohp0", "ohp1", "ohp2", "ohp3", "ohp4", "ohp5", "ohp6", 
             "ohp7", "ohp8", "squat0", "squat1", "squat2", "squat3", "squat4", "squat5", "squat6", "squat7",
             "squat8"]

workout_loc = "/Users/jamesperalta/Desktop/classes/CPSC-502/ResearchProposal/WorkoutRecognitionThesis/jupyter-notebooks/increase-decrease-results/workout_preds/"
phase_loc = "/Users/jamesperalta/Desktop/classes/CPSC-502/ResearchProposal/WorkoutRecognitionThesis/jupyter-notebooks/increase-decrease-results/phase_preds_2/"
workout_file_names = os.listdir(workout_loc)
phase_file_names = os.listdir(phase_loc)


In [49]:
all_labels = []
for file in file_names:
    workout_path = workout_loc + list(filter(lambda x: file in x, workout_file_names))[0]
    phase_path = phase_loc + list(filter(lambda x: file in x, phase_file_names))[0]

    workout_df = pd.read_csv(workout_path)
    workout_df = workout_df.rename(columns={'Unnamed: 0': 'frame'})
    
    phase_df = pd.read_csv(phase_path)
    phase_df = phase_df.rename(columns={'Unnamed: 0': 'frame'})
    
    merged_df = pd.merge(left=workout_df, right=phase_df, left_on='frame', right_on='frame')
    merged_df['video_name'] = file
    all_labels.append(merged_df)

combined_df = pd.concat(all_labels)

In [50]:
def convert_nothing_detected_to_stationary(row):
    workout_detected = row["workout_detected"]
    
    # Because were only tracking when a person is working out
    if workout_detected == "nothing":
        row["predicted phase"] = "stationary"
    
    row["label_y"] = row["label_y"].lower()
    
    return row

In [51]:
combined_df = combined_df.apply(convert_nothing_detected_to_stationary, axis=1)

In [52]:
combined_df

,frame,workout_detected,label_x,predicted phase,label_y,video_name
0,0,nothing,stationary,stationary,stationary,nothing0
1,1,nothing,stationary,stationary,stationary,nothing0
2,2,nothing,stationary,stationary,stationary,nothing0
3,3,nothing,stationary,stationary,stationary,nothing0
4,4,nothing,stationary,stationary,stationary,nothing0
...,...,...,...,...,...,...
576,576,squat,stationary,stationary,stationary,squat8
577,577,squat,stationary,stationary,stationary,squat8
578,578,squat,stationary,stationary,stationary,squat8
579,579,squat,stationary,stationary,stationary,squat8


## Split into train and test splits

In [53]:
train_set = ["nothing1", "ohp6", "squat6", "ohp2", "ohp3", "squat0", "squat1", "squat5"]
train_df = combined_df[combined_df['video_name'].isin(train_set)].reset_index(drop=True) 
test_df = combined_df[~combined_df['video_name'].isin(train_set)].reset_index(drop=True) 

## Evaluate test split results

In [54]:
train_df

,frame,workout_detected,label_x,predicted phase,label_y,video_name
0,0,nothing,stationary,stationary,stationary,nothing1
1,1,nothing,stationary,stationary,stationary,nothing1
2,2,nothing,stationary,stationary,stationary,nothing1
3,3,nothing,stationary,stationary,stationary,nothing1
4,4,nothing,stationary,stationary,stationary,nothing1
...,...,...,...,...,...,...
5161,569,squat,decreasing,decreasing,decreasing,squat6
5162,570,squat,decreasing,decreasing,decreasing,squat6
5163,571,squat,decreasing,decreasing,decreasing,squat6
5164,572,squat,decreasing,decreasing,decreasing,squat6


In [55]:
# Get train predictions and ground truths
train_pred = train_df["predicted phase"]
train_truth = train_df["label_y"]

# Get test predictions and ground truths
test_pred = test_df["predicted phase"]
test_truth = test_df["label_y"]

train_acc = accuracy_score(train_truth, train_pred)
test_acc = accuracy_score(test_truth, test_pred)

print("Training accuracy: {}".format(train_acc))
print("Testing accuracy: {}".format(test_acc))

Training accuracy: 0.7982965543941154
Testing accuracy: 0.8246640658864326


In [56]:
wrong = 0
right = 0
total = 0
wrong_indices = []
for pred in zip(train_pred, train_truth):
    y = pred[0]
    y_hat = pred[1]
    
    if y != y_hat:
        wrong += 1
        wrong_indices.append(total)
    else:
        right += 1
        
    total += 1

In [57]:
train_wrong_preds = train_df.iloc[wrong_indices]

In [58]:
train_wrong_preds

,frame,workout_detected,label_x,predicted phase,label_y,video_name
242,242,ohp,stationary,decreasing,stationary,nothing1
420,420,ohp,stationary,decreasing,stationary,nothing1
421,421,ohp,stationary,decreasing,stationary,nothing1
422,422,ohp,stationary,decreasing,stationary,nothing1
801,70,ohp,increasing,decreasing,increasing,ohp2
...,...,...,...,...,...,...
5155,563,squat,increasing,stationary,increasing,squat6
5156,564,squat,increasing,stationary,increasing,squat6
5157,565,squat,increasing,stationary,increasing,squat6
5158,566,squat,increasing,stationary,increasing,squat6


In [59]:
train_df["video_name"].value_counts()

ohp2        799
nothing1    731
squat1      722
ohp6        636
ohp3        603
squat5      584
squat6      574
squat0      517
Name: video_name, dtype: int64

In [60]:
train_wrong_preds["video_name"].value_counts()

ohp6        185
squat5      182
ohp3        175
squat1      154
squat6      137
squat0      121
ohp2         84
nothing1      4
Name: video_name, dtype: int64

In [28]:
wrong/total

0.3058459156020132

In [17]:
train_acc

0.6941540843979869